In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, log_loss, matthews_corrcoef, roc_auc_score

# Modelos de clasificación
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import kagglehub
from copy import deepcopy

# Explicación de los algoritmos

## SVM(Support Vector Machine)

Es un algoritmo de aprendisaje supervisado que se usa para los modelos de clasificación y de regresión lineal. Crea un hiperplano(recta) que separe los dos conjuntos de datos con la mayor separción posible como se puede ver en la siguiente ![GitHub Logo](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Svm_separating_hyperplanes_%28SVG%29.svg/277px-Svm_separating_hyperplanes_%28SVG%29.svg.png)
En la imagen anterior se cogería el h3 porque el h1 no los divide bien y el h2 tiene menor separación

## Regresión Logística

La regresión logistica es un modelo que sirve para predecir si un dato pertenece a una ctegoría o a otra en unfucionede unas variables independientes, se diferencia de una regresion lineal en que la logistica te da un nos porcentajes mientras que la lineal te da números.

$$
P(Y=1) = \frac{1}{1 + e^{-(b_0 + b_1X_1 + b_2X_2 + ... + b_nX_n)}}
$$

Donde:

* P(Y=1)P(Y=1) es la probabilidad de que algo ocurra (por ejemplo, que compre el producto).
*    b0 es el término de sesgo (como un punto de inicio).
*    b1​,b2​,...,bn​ son los pesos de cada variable (edad, salario, etc.).
*    X1​,X2​,...,Xn​ son los valores de esas variables para cada persona.
*    e es una constante matemática (~2.718).


## Arbol de decisión

Como se puede ver en la siguiente imagen un arbol de decisión es un modelo de aprendizaje en que se forman diagramas de construcciones lógicas

![GitHub Logo](https://upload.wikimedia.org/wikipedia/commons/e/e6/Arbol_de_decision.png)

## Bosques aleatorios 

Un modelo de bosque aleatorio son varios arbloes de decisón puestos juntos, cada uno es entrenado con un parte de los datos y  despues hace que voten para escoger la mejor decisón
![GitHub Logo](https://miro.medium.com/v2/resize:fit:640/format:webp/0*Z52NbkU3Uddq3gX7.png)

## kNN, o el algoritmo de k vecino más cercano 

Es un modelo de clasificación que mira los k vecinos más cercanos donde K es el número de vecinos. Las ecuaciones más usadas para calcular la distacia del punto con los más cercanas son la euclidiana y la Manhattan, puedes usar la ecuación de Minkowski para poder usar las dos cambiando solo un parámetro

#### **Distancia Euclidiana**
$$ d(A, B) = \sqrt{\sum_{i=1}^{n} (x_i^{(A)} - x_i^{(B)})^2} $$

#### **Distancia de Manhattan**
$$ d(A, B) = \sum_{i=1}^{n} |x_i^{(A)} - x_i^{(B)}| $$

#### **Distancia de Minkowski**
$$ d(A, B) = \left( \sum_{i=1}^{n} |x_i^{(A)} - x_i^{(B)}|^p \right)^{\frac{1}{p}} $$

# Aplicación de los algoritmos

## Algoritmos sin hyperparametros

Ponemos un RANDOM_STATE para a la hora de comprar los modelos tengan todos la misma semilla

In [32]:
RANDOM_STATE = 6

Cargamos los datasets

In [33]:
breast_cancer_wisconsin_diagnostic_dataset_path = kagglehub.dataset_download('utkarshx27/breast-cancer-wisconsin-diagnostic-dataset')
wine_quality_dataset_path = kagglehub.dataset_download('yasserh/wine-quality-dataset')
accidentes_de_trfico_de_madrid_espaa_2019_a_2023_path = kagglehub.dataset_download('jairoordezpacheco/accidentes-de-trfico-de-madrid-espaa-2019-a-2023')
phishing_website_detector_path = kagglehub.dataset_download('eswarchandt/phishing-website-detector')

brca = pd.read_csv(breast_cancer_wisconsin_diagnostic_dataset_path+'/brca.csv')
wine = pd.read_csv(wine_quality_dataset_path+'/WineQT.csv')
phishing= pd.read_csv(phishing_website_detector_path+'/phishing.csv')
accidents = pd.read_csv(accidentes_de_trfico_de_madrid_espaa_2019_a_2023_path+'/datos_madrid.csv',encoding='MacRoman')

Preparamos los datos para pasarselos a los modelos

In [34]:
def preprocess_brca(df):
    X = df.drop(columns=['y'])
    y = df['y'].apply(lambda x: 1 if x == 'M' else 0)
    return X, y

In [35]:
def preprocess_wine(df):
    df['quality'] = df['quality'].apply(lambda x: 1 if x > 5 else 0)
    X = df.drop(columns=['quality', 'Id'])
    y = df['quality']
    return X, y

In [36]:
def preprocess_phishing(df):
    X = df.drop(columns=['class', 'Index'])
    y = df['class'].apply(lambda x: 1 if x == 1 else 0)
    return X, y

In [37]:
def preprocess_accidents(df):
    df = df.drop(columns=['localizacion','numero','distrito','lesividad'])
    df = df.dropna()

# Mapeo de códigos a 0 (leve) o 1 (grave)
    mapa = {
        0: 0,  # LEVE
        1: 0,  # LEVE
        2: 0,  # LEVE
        3: 1,  # GRAVE
        4: 1,  # FALLECIDO (considerado grave)
        5: 0,  # LEVE
        6: 0,  # LEVE
        7: 0,  # LEVE
        14: 0, # Sin asistencia -> 0
        77: 0  # Se desconoce -> 0
    }


    df['cod_lesividad'] = df['cod_lesividad'].map(mapa)

    le = LabelEncoder()
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = le.fit_transform(df[col])
    df.to_csv('accidents2.csv', index=False)        
    X = df.drop(columns=['cod_lesividad'])
    y = df['cod_lesividad']
    return X, y

In [38]:
X_brca, y_brca = preprocess_brca(brca)
X_wine, y_wine = preprocess_wine(wine)
X_phishing, y_phishing = preprocess_phishing(phishing)
X_accidents, y_accidents = preprocess_accidents(accidents.head(20000))

Separamos entre datos de test y de entrnamiento

In [39]:
X_train_brca, X_test_brca, y_train_brca, y_test_brca = train_test_split(X_brca, y_brca, test_size=0.2, random_state=RANDOM_STATE)
X_train_wine, X_test_wine, y_train_wine, y_test_wine = train_test_split(X_wine, y_wine, test_size=0.2, random_state=RANDOM_STATE)
X_train_phishing, X_test_phishing, y_train_phishing, y_test_phishing = train_test_split(X_phishing, y_phishing, test_size=0.2, random_state=RANDOM_STATE)
X_train_accidents, X_test_accidents, y_train_accidents, y_test_accidents = train_test_split(X_accidents, y_accidents, test_size=0.2, random_state=RANDOM_STATE)

Escalamos los dato de entrada de todos los datasets menos del de phishing que viene ya normalizado

In [40]:
scaler = StandardScaler()
X_train_brca = scaler.fit_transform(X_train_brca)
X_test_brca = scaler.transform(X_test_brca)
X_train_wine = scaler.fit_transform(X_train_wine)
X_test_wine = scaler.transform(X_test_wine)
X_train_accidents = scaler.fit_transform(X_train_accidents)
X_test_accidents = scaler.transform(X_test_accidents)

Creamos el metodo para evaluar los modelos con todas las metricas más dos nuevas que a continuación explicaremos.

ROC AUC :La métrica ROC AUC mide la capacidad de un modelo para diferenciar entre clases. Específicamente, es el área bajo la curva ROC, que es un gráfico que muestra la relación entre la tasa de verdaderos positivos (TPR) y la tasa de falsos positivos (FPR) a diferentes umbrales de clasificación.

Matriz de Confusión La matriz de confusión es una tabla que se utiliza para describir el rendimiento de un modelo de clasificación. Muestra el número de predicciones correctas e incorrectas desglosadas por cada clase.

Componentes de la Matriz de Confusión:
True Positives (TP): Casos positivos correctamente clasificados.
True Negatives (TN): Casos negativos correctamente clasificados.
False Positives (FP): Casos negativos incorrectamente clasificados como positivos.
False Negatives (FN): Casos positivos incorrectamente clasificados como negativos.

[[TN, FP],

 [FN, TP]]

In [41]:
modelos={}
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    modelos[model_name] = deepcopy(model)
    return accuracy, precision, recall, f1, roc_auc, cm

Cargamos los modelos

In [42]:
svm = SVC(random_state=RANDOM_STATE)
log_reg = LogisticRegression(random_state=RANDOM_STATE)
decision_tree = DecisionTreeClassifier(random_state=RANDOM_STATE)
random_forest = RandomForestClassifier(random_state=RANDOM_STATE)
knn = KNeighborsClassifier()

Preparamos los modelos y los datos

In [43]:
datasets = {
    'BRCA': (X_train_brca, X_test_brca, y_train_brca, y_test_brca),
    'Wine': (X_train_wine, X_test_wine, y_train_wine, y_test_wine),
    'Phishing': (X_train_phishing, X_test_phishing, y_train_phishing, y_test_phishing),
    'Accidents': (X_train_accidents, X_test_accidents, y_train_accidents, y_test_accidents)
}

In [44]:
models = {
    'SVM': svm,
    'Logistic Regression': log_reg,
    'Decision Tree': decision_tree,
    'Random Forest': random_forest,
    'kNN': knn
}

Ejecutamos los modelos con los datos de los datasets y los guardamos en un diccionario para poder mostralos a continuación

In [45]:
results = {}

for dataset_name, (X_train, X_test, y_train, y_test) in datasets.items():
    results[dataset_name] = {}
    for model_name, model in models.items():
        accuracy, precision, recall, f1, roc_auc, cm = evaluate_model(model, X_train, X_test, y_train, y_test, dataset_name+"_"+model_name)
        results[dataset_name][model_name] = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1,
            'ROC AUC': roc_auc,
            'Confusion Matrix': cm
        }

c:\Users\angel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\angel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\angel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [46]:
for dataset_name, dataset_results in results.items():
    print(f"Results for {dataset_name} dataset:")
    for model_name, metrics in dataset_results.items():
        print(f"  {model_name}:")
        for metric_name, value in metrics.items():
            print(f"    {metric_name}: {value}")
        print()

Results for BRCA dataset:
  SVM:
    Accuracy: 0.9912280701754386
    Precision: 1.0
    Recall: 0.9791666666666666
    F1 Score: 0.9894736842105263
    ROC AUC: 0.9895833333333333
    Confusion Matrix: [[66  0]
 [ 1 47]]

  Logistic Regression:
    Accuracy: 0.9912280701754386
    Precision: 1.0
    Recall: 0.9791666666666666
    F1 Score: 0.9894736842105263
    ROC AUC: 0.9895833333333333
    Confusion Matrix: [[66  0]
 [ 1 47]]

  Decision Tree:
    Accuracy: 1.0
    Precision: 1.0
    Recall: 1.0
    F1 Score: 1.0
    ROC AUC: 1.0
    Confusion Matrix: [[66  0]
 [ 0 48]]

  Random Forest:
    Accuracy: 1.0
    Precision: 1.0
    Recall: 1.0
    F1 Score: 1.0
    ROC AUC: 1.0
    Confusion Matrix: [[66  0]
 [ 0 48]]

  kNN:
    Accuracy: 0.9736842105263158
    Precision: 0.9787234042553191
    Recall: 0.9583333333333334
    F1 Score: 0.968421052631579
    ROC AUC: 0.9715909090909092
    Confusion Matrix: [[65  1]
 [ 2 46]]

Results for Wine dataset:
  SVM:
    Accuracy: 0.7860262008

### Conclusión de BRCA dataset:

* SVM y Logistic Regression: Ambos modelos tienen un rendimiento muy alto, con precisión perfecta  (De todos los casos que los modelos predijeron como positivos, el 100% eran realmente positivos) y un recall ligeramente menor(Los dos modelos fallaron y tu vieron 1 falso negativo), lo que indica que clasifican correctamente casi todos los casos positivos y negativos.
* Decision Tree y Random Forest: Ambos modelos tienen un rendimiento perfecto en todas las métricas, por lo que seguramente esten sobreajustados.
* kNN: Tiene un rendimiento ligeramente inferior en comparación con los otros modelos, pero aún así es bastante alto con una precisión y recall muy buenos.

* Viendo los resultado anteriormente descritos yo me quedaría con el SVM o la regresión logistica porque no están sobreajustados, para escoger alguno habría que ver el rendimiento de cada uno de los dos modelos y cual gasta más

### Conclusión de Wine dataset:
* SVM y Random Forest: Ambos modelos tienen el mejor rendimiento en este dataset, con SVM ligeramente inferior a Random Forest en precisión y recall.
* Logistic Regression y Decision Tree: Tienen un rendimiento aceptable, pero inferior a SVM y Random Forest. Decision Tree tiene un recall más bajo, lo que indica que no clasifica tan bien los casos positivos.
* kNN: Tiene el rendimiento más bajo en comparación con los otros modelos, con una precisión y recall más bajos.
* Viendo los resultados anteriormente descritos yo me quedaría con random forest porque es ligeramente superior al SVM

### Conclusión de Phishing Dataset
* Random Forest: Tiene el mejor rendimiento en este dataset, con la mayor precisión y recall, lo que indica que clasifica correctamente la mayoría de los casos positivos y negativos.
* Decision Tree y SVM: Tienen un rendimiento muy alto, con precisión y recall ligeramente inferiores a Random Forest.
* Logistic Regression: Tiene un rendimiento aceptable, pero inferior a Random Forest, Decision Tree y SVM.
kNN: Tiene un rendimiento similar a SVM, con una precisión y recall muy buenos.
* Viendo los resultados anteriormente descritos yo me quedaría con random forest porque es claramente superior a los demás

### Conclusión Accidents dataset

* SVM, Logistic Regression y Random Forest: Todos estos modelos tienen una alta precisión en términos de clasificar los negativos, pero no identifican correctamente ningún caso positivo, lo que se refleja en las métricas de precisión, recall y F1 score de 0.
* Decision Tree: Tiene un rendimiento ligeramente mejor en términos de identificar casos positivos, pero aún así es bastante bajo.
* kNN: Similar a SVM, Logistic Regression y Random Forest, no identifica correctamente ningún caso positivo.
* Viendo los resultado anteriormente descritos, aunque son datos pobres el mejor modelo es el arbo,l de decisió que detecta algun positivo

## Algoritmos con hyperparametros

Cargamos los modelos con los hiperparámetros

Primero empezamos por el SVM y he usado los siguientes hiperparámtros:

* C: Controla cuánto permitimos que los puntos estén en el lado incorrecto de la línea. Un valor alto de C intenta clasificar todos los puntos correctamente, mientras que un valor bajo permite más errores pero puede generalizar mejor. El valor por defecto es 1
* kernel: Define cómo se transforman los datos para encontrar la mejor línea de separación. Puede ser lineal o no lineal, en nuestro caso hemos usado el kernel linal que se usa cuando los datos e pueden dividir facilmente por una linea y el rbf que se usa cuando hay datos que no se pueden separar fácilmente con una recta y lo que hace es crear un espacio de una dimensión mayor para poder dividirlo por una linea.
* tol: Es la tolerancia para el criterio de parada. Controla cuándo el algoritmo debe detenerse. Un valor más pequeño significa que el algoritmo se detendrá solo cuando los cambios sean muy pequeños, lo que puede llevar a un ajuste más preciso pero también a un tiempo de entrenamiento más largo. El valor por defecto es 1e-3

In [47]:
# Inicializar los modelos con un hiperparámetro específico por ejecución
svm_C_10 = SVC(C=10.0, random_state=RANDOM_STATE)
svm_C_0_1 = SVC(C=0.1, random_state=RANDOM_STATE)
svm_kernel_linear = SVC(kernel='linear', random_state=RANDOM_STATE)
svm_kernel_rbf = SVC(kernel='rbf', random_state=RANDOM_STATE)
svm_tol_1e_0_1 = SVC(tol=0.1, random_state=RANDOM_STATE)
svm_tol_1e_6 = SVC(tol=1e-6, random_state=RANDOM_STATE) 

En la regresioón logistica he usado los siguientes hiperparámetros:

* C: Controla cuánto penalizamos la complejidad del modelo. Un valor alto de C intenta ajustar el modelo muy bien a los datos, mientras que un valor bajo lo hace más simple y puede generalizar mejor a nuevos datos. El valor por defecto es 1

* solver: Es el método que se usa para encontrar los mejores valores para el modelo. Por ejemplo, 'liblinear' es bueno para conjuntos de datos pequeños, mientras que 'lbfgs' es más adecuado para conjuntos de datos más grandes.

* max_iter: Es el número máximo de veces que el método intentará ajustar el modelo. Si el modelo no converge antes de alcanzar este número, se detendrá.

In [48]:
log_reg_C_1_0 = LogisticRegression(C=10.0, random_state=RANDOM_STATE)
log_reg_C_0_1 = LogisticRegression(C=0.1, random_state=RANDOM_STATE)
log_reg_solver_liblinear = LogisticRegression(solver='liblinear', random_state=RANDOM_STATE)
log_reg_solver_lbfgs = LogisticRegression(solver='lbfgs', random_state=RANDOM_STATE)
log_reg_max_iter_200 = LogisticRegression(max_iter=200, random_state=RANDOM_STATE)
log_reg_max_iter_100 = LogisticRegression(max_iter=100, random_state=RANDOM_STATE)


En los arboles de decisión he usado los siguiente hiperparámetros:

* max_depth: La profundidad máxima del árbol, es decir, cuántas preguntas puede hacer antes de llegar a una decisión. Un valor más alto puede llevar a un modelo más complejo y específico, mientras que un valor más bajo puede hacer que el modelo sea más simple y general. 

* min_samples_split: El número mínimo de datos necesarios para hacer una nueva pregunta. Si un nodo tiene menos datos que este número, no se dividirá. Por defecto está puesto en 2

* min_samples_leaf: El número mínimo de datos necesarios para que una hoja sea válida. Esto ayuda a evitar que el modelo se ajuste demasiado a los datos de entrenamiento. Por defecto está puesto en 1

In [49]:
decision_tree_max_depth_10 = DecisionTreeClassifier(max_depth=10, random_state=RANDOM_STATE)
decision_tree_max_depth_5 = DecisionTreeClassifier(max_depth=5, random_state=RANDOM_STATE)
decision_tree_min_samples_split_8 = DecisionTreeClassifier(min_samples_split=8, random_state=RANDOM_STATE)
decision_tree_min_samples_split_4 = DecisionTreeClassifier(min_samples_split=4, random_state=RANDOM_STATE)
decision_tree_min_samples_leaf_0_5 = DecisionTreeClassifier(min_samples_leaf=0.5, random_state=RANDOM_STATE)
decision_tree_min_samples_leaf_5 = DecisionTreeClassifier(min_samples_leaf=5, random_state=RANDOM_STATE)


En los bosques aleatorios he usado los siguiente hiperparámetros:

* n_estimators: El número de árboles en el bosque. Más árboles pueden llevar a un modelo más preciso, pero también aumentan el tiempo de entrenamiento. Por defecto está puesto en 1

* max_depth: La profundidad máxima de cada árbol. Controla cuántas preguntas puede hacer cada árbol antes de llegar a una decisión.

* min_samples_split: El número mínimo de datos necesarios para hacer una nueva pregunta en cada árbol. Ayuda a controlar la complejidad de los árboles individuales. Por defecto está puesto en 2

In [50]:
random_forest_n_estimators_200 = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE)
random_forest_n_estimators_100 = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
random_forest_max_depth_10 = RandomForestClassifier(max_depth=10, random_state=RANDOM_STATE)
random_forest_max_depth_5 = RandomForestClassifier(max_depth=5, random_state=RANDOM_STATE)
random_forest_min_samples_split_8 = RandomForestClassifier(min_samples_split=8, random_state=RANDOM_STATE)
random_forest_min_samples_split_4 = RandomForestClassifier(min_samples_split=4, random_state=RANDOM_STATE)


* n_neighbors: El número de puntos cercanos a considerar para tomar la decisión. Un valor más alto puede hacer que el modelo sea más general, mientras que un valor más bajo puede hacer que el modelo sea más específico. Por defecto está puesto en 5

* weights: Cómo se ponderan los puntos cercanos. 'uniform' significa que todos los puntos tienen el mismo peso, mientras que 'distance' significa que los puntos más cercanos tienen más peso. por defecto está puesto en uniform

* metric: La forma en que se mide la distancia entre los puntos. Por ejemplo, 'euclidean' para la distancia euclidiana (línea recta) y 'manhattan' para la distancia de manhattan (suma de las distancias horizontales y verticales). Por defecto está puesto en minkowski

In [51]:
knn_n_neighbors_2 = KNeighborsClassifier(n_neighbors=2)
knn_n_neighbors_10 = KNeighborsClassifier(n_neighbors=10)
knn_weights_distance = KNeighborsClassifier(weights='distance')
knn_weights_uniform = KNeighborsClassifier(weights='uniform')
knn_metric_euclidean = KNeighborsClassifier(metric='euclidean')
knn_metric_manhattan = KNeighborsClassifier(metric='manhattan')

Preparamops los modelos

In [52]:
models_with_params = {
    'SVM_C_10': svm_C_10,
    'SVM_C_0_1': svm_C_0_1,
    'SVM_kernel_linear': svm_kernel_linear,
    'SVM_kernel_rbf': svm_kernel_rbf,
    'SVM_tol_1e_3': svm_tol_1e_0_1,
    'SVM_tol_1e_4': svm_tol_1e_6,
    'Logistic Regression_C_1': log_reg_C_1_0,
    'Logistic Regression_C_0_1': log_reg_C_0_1,
    'Logistic Regression_solver_liblinear': log_reg_solver_liblinear,
    'Logistic Regression_solver_lbfgs': log_reg_solver_lbfgs,
    'Logistic Regression_max_iter_200': log_reg_max_iter_200,
    'Logistic Regression_max_iter_100': log_reg_max_iter_100,
    'Decision Tree_max_depth_10': decision_tree_max_depth_10,
    'Decision Tree_max_depth_5': decision_tree_max_depth_5,
    'Decision Tree_min_samples_split_4': decision_tree_min_samples_split_8,
    'Decision Tree_min_samples_split_2': decision_tree_min_samples_split_4,
    'Decision Tree_min_samples_leaf_2': decision_tree_min_samples_leaf_0_5,
    'Decision Tree_min_samples_leaf_1': decision_tree_min_samples_leaf_5,
    'Random Forest_n_estimators_200': random_forest_n_estimators_200,
    'Random Forest_n_estimators_100': random_forest_n_estimators_100,
    'Random Forest_max_depth_10': random_forest_max_depth_10,
    'Random Forest_max_depth_5': random_forest_max_depth_5,
    'Random Forest_min_samples_split_4': random_forest_min_samples_split_8,
    'Random Forest_min_samples_split_2': random_forest_min_samples_split_4,
    'kNN_n_neighbors_3': knn_n_neighbors_2,
    'kNN_n_neighbors_5': knn_n_neighbors_10,
    'kNN_weights_distance': knn_weights_distance,
    'kNN_weights_uniform': knn_weights_uniform,
    'kNN_metric_euclidean': knn_metric_euclidean,
    'kNN_metric_manhattan': knn_metric_manhattan
}

Ejecutamos los modelos con los datos de los hiperparametros y los guardamos en un diccionario para poder mostralos a continuación

In [53]:
results_with_params = {}

for dataset_name, (X_train, X_test, y_train, y_test) in datasets.items():
    results_with_params[dataset_name] = {}
    for model_name, model in models_with_params.items():
        accuracy, precision, recall, f1, roc_auc, cm = evaluate_model(model, X_train, X_test, y_train, y_test, dataset_name+"_"+model_name)
        results_with_params[dataset_name][model_name] = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1,
            'ROC AUC': roc_auc,
            'Confusion Matrix': cm
        }

c:\Users\angel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\angel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\angel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\angel\anaconda3\Lib\site-packages\sklearn\metrics\_clas

In [54]:
for dataset_name, dataset_results in results_with_params.items():
    print(f"Results for {dataset_name} dataset with hyperparameters:")
    for model_name, metrics in dataset_results.items():
        print(f"  {model_name}:")
        for metric_name, value in metrics.items():
            print(f"    {metric_name}: {value}")
        print()

Results for BRCA dataset with hyperparameters:
  SVM_C_10:
    Accuracy: 0.9912280701754386
    Precision: 1.0
    Recall: 0.9791666666666666
    F1 Score: 0.9894736842105263
    ROC AUC: 0.9895833333333333
    Confusion Matrix: [[66  0]
 [ 1 47]]

  SVM_C_0_1:
    Accuracy: 0.956140350877193
    Precision: 1.0
    Recall: 0.8958333333333334
    F1 Score: 0.945054945054945
    ROC AUC: 0.9479166666666667
    Confusion Matrix: [[66  0]
 [ 5 43]]

  SVM_kernel_linear:
    Accuracy: 0.9824561403508771
    Precision: 1.0
    Recall: 0.9583333333333334
    F1 Score: 0.9787234042553191
    ROC AUC: 0.9791666666666667
    Confusion Matrix: [[66  0]
 [ 2 46]]

  SVM_kernel_rbf:
    Accuracy: 0.9912280701754386
    Precision: 1.0
    Recall: 0.9791666666666666
    F1 Score: 0.9894736842105263
    ROC AUC: 0.9895833333333333
    Confusion Matrix: [[66  0]
 [ 1 47]]

  SVM_tol_1e_3:
    Accuracy: 0.9912280701754386
    Precision: 1.0
    Recall: 0.9791666666666666
    F1 Score: 0.9894736842105263

### Conclusión de BRCA dataset:
Modelos SVM:

* SVM_C_10 y SVM_kernel_rbf: Ambos modelos tienen un rendimiento muy alto, con precisión perfecta y un recall ligeramente menor, lo que indica que clasifican correctamente casi todos los casos positivos y negativos.
* SVM_C_0_1: Tiene un rendimiento ligeramente inferior, con un recall más bajo, lo que indica que no clasifica tan bien los casos positivos.

Modelos de Regresión Logística:

* Todos los modelos de regresión logística tienen un rendimiento muy alto, con precisión perfecta y un recall ligeramente menor, similar a los mejores modelos SVM.

Modelos de Árbol de Decisión:

* Decision Tree_max_depth_10, Decision Tree_max_depth_5, Decision Tree_min_samples_split_4, Decision Tree_min_samples_split_2, Decision Tree_min_samples_leaf_1: Todos estos modelos tienen un rendimiento perfecto en todas las métricas, lo que sugiere que pueden estar sobreajustados a los datos de entrenamiento.
*Decision Tree_min_samples_leaf_2: Tiene un rendimiento muy bajo, no identificando correctamente ningún caso positivo.
Modelos de Bosque Aleatorio:

* Todos los modelos de bosque aleatorio tienen un rendimiento perfecto en todas las métricas, similar a los mejores modelos de árbol de decisión.

Modelos kNN:

* kNN_n_neighbors_3: Tiene un rendimiento ligeramente inferior en comparación con los otros modelos, con un recall más bajo.
* kNN_n_neighbors_5, kNN_weights_distance, kNN_weights_uniform, kNN_metric_euclidean: Tienen un rendimiento muy alto, con precisión y recall muy buenos.
* kNN_metric_manhattan: Tiene un rendimiento ligeramente inferior, con un recall más bajo.

Viendo los datos anteriores yo no escogería ni los random forest ni los arboles de desición porque da todo 1 y estarán sobreajustados lo que haran que no sean buenos con los datos de prueba por lo que scogeré el SVM con kernel RBF, podría haber escogido otros con los mismo resultados como Logistic Regression_C_0_1 o SVM_tol_1e_4 pero me he decantado por este

### Conclusión de Wine Dataset:

Modelos SVM:

* SVM_kernel_rbf: Tiene el mejor rendimiento entre los modelos SVM, con una alta precisión y recall, lo que indica un buen equilibrio entre la clasificación de casos positivos y negativos.

Modelos de Regresión Logística:

* Logistic Regression_C_0_1: Tiene un rendimiento ligeramente superior en comparación con otros modelos de regresión logística, con un buen equilibrio entre precisión y recall.

Modelos de Árbol de Decisión:

* Decision Tree_min_samples_split_4: Tiene el mejor rendimiento entre los modelos de árbol de decisión, con una precisión y recall aceptables.

Modelos de Bosque Aleatorio:

* Random Forest_min_samples_split_4: Tiene el mejor rendimiento entre los modelos de bosque aleatorio, con una alta precisión y recall, lo que indica un buen equilibrio entre la clasificación de casos positivos y negativos.

Modelos kNN:

* kNN_weights_distance: Tiene el mejor rendimiento entre los modelos kNN, con una alta precisión y recall, lo que indica un buen equilibrio entre la clasificación de casos positivos y negativos.

Viendo los datos anteriores el mejor modelo es el Random Forest_min_samples_split_4


### Conclusión de Phishing dataset:

Modelos SVM:

SVM_C_10: Tiene el mejor rendimiento entre los modelos SVM, con una alta precisión y recall, lo que indica un buen equilibrio entre la clasificación de casos positivos y negativos.
Modelos de Regresión Logística:

Logistic Regression_C_0_1: Tiene un rendimiento ligeramente superior en comparación con otros modelos de regresión logística, con un buen equilibrio entre precisión y recall.
Modelos de Árbol de Decisión:

Decision Tree_min_samples_split_2: Tiene el mejor rendimiento entre los modelos de árbol de decisión, con una alta precisión y recall, lo que indica un buen equilibrio entre la clasificación de casos positivos y negativos.
Modelos de Bosque Aleatorio:

Random Forest_n_estimators_100: Tiene el mejor rendimiento entre los modelos de bosque aleatorio, con una alta precisión y recall, lo que indica un buen equilibrio entre la clasificación de casos positivos y negativos.
Modelos kNN:

kNN_weights_distance: Tiene el mejor rendimiento entre los modelos kNN, con una alta precisión y recall, lo que indica un buen equilibrio entre la clasificación de casos positivos y negativos.

Viendo los datos anteriores el mejor modelo es el Random Random Forest_n_estimators_100


### Conclusión de Phishing dataset:

* Modelos SVM y Regresión Logística:

Todos los modelos SVM y de regresión logística tienen una alta precisión en términos de clasificar los negativos, pero no identifican correctamente ningún caso positivo, lo que se refleja en las métricas de precisión, recall y F1 score de 0.

Modelos de Árbol de Decisión:

* Decision Tree_max_depth_10: Tiene un rendimiento ligeramente mejor en términos de identificar casos positivos, pero aún así es bastante bajo.
* Decision Tree_min_samples_split_4: Tiene un rendimiento ligeramente mejor en términos de identificar casos positivos, pero aún así es bastante bajo.

Modelos de Bosque Aleatorio:

* Todos los modelos de bosque aleatorio tienen una alta precisión en términos de clasificar los negativos, pero no identifican correctamente ningún caso positivo.

Modelos kNN:

* kNN_n_neighbors_3: Tiene un rendimiento ligeramente mejor en términos de identificar casos positivos, pero aún así es bastante bajo.

Viendo los datos anteriores el mejor modelo es el Random Random Decision Tree_max_depth_10


## Datasets de prueba

Creamos los datasets de prueba

In [55]:
def create_example_dataset(x, y):
    positive_indices = y[y == 1].index[:5]
    negative_indices = y[y == 0].index[:5]
    example_indices = positive_indices.union(negative_indices)
    X_example = x.loc[example_indices]
    y_example = y.loc[example_indices]
    return X_example, y_example

Los dividimos

In [ ]:
X_example_brca, y_example_brca = create_example_dataset(X_brca, y_brca)
X_example_wine, y_example_wine = create_example_dataset(X_wine, y_wine)
X_example_accidents, y_example_accidents = create_example_dataset(X_accidents, y_accidents)
X_example_phishing, y_example_phishing = create_example_dataset(X_phishing, y_phishing)

Los escalamos

In [58]:
# Escalar los datos
scaler = StandardScaler()
X_example_brca = scaler.fit_transform(X_example_brca)
X_example_wine = scaler.fit_transform(X_example_wine)
X_example_accidents = scaler.fit_transform(X_example_accidents)

Preparamos los diccionarios con los datos

In [59]:
datasets = {
    'BRCA': (X_example_brca, y_example_brca),
    'Wine': (X_example_wine, y_example_wine),
    'Phishing': (X_example_phishing, y_example_phishing),
    'Accidents': (X_example_accidents, y_example_accidents)    
}

modelos_prueba = {
    'BRCA': 'SVM_kernel_rbf',
    'Wine': 'Random Forest_min_samples_split_4',
    'Phishing': 'Random Forest_n_estimators_100',
    'Accidents': 'Decision Tree_max_depth_10'
}


Hacems las preddciones con los modelos seleccionados, los evaluamos y los mostramos

In [56]:
def evaluate_model_prueba(model, X_test, y_test):
    print(model)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return accuracy, precision, recall, f1, roc_auc, cm

In [60]:
results_prueba = {}
for dataset_name, (x, y) in datasets.items():
    model = modelos[dataset_name+"_"+modelos_prueba[dataset_name]]

    accuracy, precision, recall, f1, roc_auc, cm = evaluate_model_prueba(model, x, y)
    results_prueba[dataset_name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROC AUC': roc_auc,
        'Confusion Matrix': cm
    }

SVC(random_state=6)
RandomForestClassifier(min_samples_split=8, random_state=6)
RandomForestClassifier(random_state=6)
DecisionTreeClassifier(max_depth=10, random_state=6)


c:\Users\angel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [61]:
for dataset_name, metrics in results_prueba.items():
    print(f"Results for {dataset_name} dataset with hyperparameters:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value}")
    print()

Results for BRCA dataset with hyperparameters:
  Accuracy: 1.0
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
  ROC AUC: 1.0
  Confusion Matrix: [[5 0]
 [0 5]]

Results for Wine dataset with hyperparameters:
  Accuracy: 0.7
  Precision: 0.6666666666666666
  Recall: 0.8
  F1 Score: 0.7272727272727273
  ROC AUC: 0.7000000000000001
  Confusion Matrix: [[3 2]
 [1 4]]

Results for Phishing dataset with hyperparameters:
  Accuracy: 1.0
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
  ROC AUC: 1.0
  Confusion Matrix: [[5 0]
 [0 5]]

Results for Accidents dataset with hyperparameters:
  Accuracy: 0.5
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0
  ROC AUC: 0.5
  Confusion Matrix: [[5 0]
 [5 0]]



* BRCA y Phishing: Los modelos para estos datasets tienen un rendimiento perfecto en el dataset de ejemplo, clasificando correctamente todas las muestras.
* Wine: El modelo tiene un rendimiento moderado, con una precisión y recall razonables, pero no perfectos.
Accidents: El modelo tiene un rendimiento pobre, con una precisión y recall de 0, indicando que no pudo identificar ninguna muestra positiva correctamente.

* Estos resultados sugieren que los modelos para BRCA y Phishing están bien ajustados para sus respectivos datasets, mientras que el modelo para Accidents necesita mejoras significativas. El modelo para Wine tiene un rendimiento aceptable, pero también podría beneficiarse de ajustes adicionales.